In [1]:
#================================================
#DEBUG TOGGLE SWITCH

def d(*args, **kwargs):
    if d.ON:
        print(*args, **kwargs)

d.ON = False          # <- module-local switch

##example usage
#d("Initial x shape:", x.shape)
#================================================



#===========================================================000

import torch
import torch.nn as nn
import torch.nn.functional as F
#import cv2 EDIT: was giving problems in inference phase
import numpy as np

#=====================================================================
# LOSS FUNCTION GETTER

def get_loss(lambda_bce=3.0):  #factory function that returns a customised loss_fn with lambda = 3.0
  ce_loss_fn = nn.CrossEntropyLoss() #this here is for the softmax head, will work on pred_softmax and target_labels (find em below)
  bce_loss_fn = nn.BCEWithLogitsLoss() #this here for the sigmoid head instead; mask on edges to be applied later

  def loss_fn(pred_softmax, pred_sigmoid, target_labels, target_objectness): #this in fact is the real loss function
    loss_ce = ce_loss_fn(pred_softmax, target_labels) #for semantic segmentation, cross entropy; WATCH OUT! target_labels are the ground truth classes

    #BCE on object boundaries
    #boundary_masks =  [] #first we gotta compute the boundary mask 1 where label edges are present; EDIT: giving problems in inference

    """ #EDIT: was giving problems in inference
    for label in target_labels.cpu().numpy():
      label_uint8 = label.astype(np.uint8)
      edge = cv2.Canny(label_uint8, 0, 1) #binary edge map, very first lesson for theory, edges have 255
      dilated = cv2.dilate(edge, np.ones((3,3), np.uint8), iterations=1) #stretches borders of 3px with 3x3 krnl
      boundary_masks.append(dilated / 255.0) #result normalization (0,255) -> (0,1)
    """

    #Stacking and conversion to a tensor
    #boundary_mask_tensor = torch.from_numpy(np.stack(boundary_masks)).to(target_objectness.device).float() #boundary mask recombined as a tensor; stack is to send it to GPU as a batch tensor; EDIT: was giving problems in inference

    #makin sure shape is [B, 1, H, W] as well as the pred_sigmoid
    #boundary_mask_tensor = boundary_mask_tensor.unsqueeze(1) #changing shape to multiply it with bce; EDIT: was giving problems in inference



    #raw pixel-wise BCE computation
    pred_sigmoid = torch.clamp(pred_sigmoid, min=-10.0, max=10.0)
    #pred_sigmoid = pred_sigmoid.squeeze(1)
    #target_objectness = target_objectness.unsqueeze(1)

    loss_bce = bce_loss_fn(pred_sigmoid, target_objectness) #pixel-wies but still unmasked

    #boundary mask application: loss only on edges
    #bce_masked = bce_raw * boundary_mask_tensor #EDIT: giving problems in inference

    #taking the mean over all the boundary pixels (shouldnt divide by total pixels)
    #loss_bce = bce_masked.sum() / (boundary_mask_tensor.sum() + 1e-2) #1e-6 needed to avoid division by 0 if batch has no edges, EDIT: 1e-6 was too high, tried to lift up at 1e-2 to see if avoids loss explosion; EDIT: was giving problems in inference

    #computing total
    loss_total = loss_ce + lambda_bce * loss_bce #combining the two to handle them together

    d("Loss cross entropy:", loss_ce.item())
    d("Loss BINARY cross entropy (object_mask):", loss_bce.item())
    d("LOSS TOTAL:", loss_total.item())

    return loss_total #to be passed to the backward pass

  return loss_fn

#========================================================================================================================================================
# DUMMY NUMERIC TEST (EDITED for full-mask objectness loss)

if __name__ == "__main__":
    d.ON = True

    # Dummy dimensions
    B, H, W = 1, 2, 2
    n_classes = 3
    lambda_value = 2.0

    # Simulated logits for semantic segmentation
    pred_softmax = torch.tensor([[[[ 4.0,  4.0],
                                   [ 4.0,  4.0]],
                                  [[-1.0, -1.0],
                                   [-1.0, -1.0]],
                                  [[-2.0, -2.0],
                                   [-2.0, -2.0]]]], dtype=torch.float32)  # shape [1, 3, 2, 2]

    # Ground truth segmentation (all class 0)
    target_labels = torch.zeros((B, H, W), dtype=torch.long)  # shape [1, 2, 2]

    # Simulated logits for objectness
    pred_sigmoid = torch.tensor([[[[3.0, 3.0],
                                   [3.0, 3.0]]]], dtype=torch.float32)  # shape [1, 1, 2, 2]

    # Ground truth objectness mask (all pixels = object)
    target_objectness = torch.ones((B, 1, H, W), dtype=torch.float32)  # shape [1, 1, 2, 2]

    # Send to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pred_softmax = pred_softmax.to(device)
    pred_sigmoid = pred_sigmoid.to(device)
    target_labels = target_labels.to(device)
    target_objectness = target_objectness.to(device)

    # Compute loss
    criterion = get_loss(lambda_bce=lambda_value)
    loss = criterion(pred_softmax, pred_sigmoid, target_labels, target_objectness)

    d(f"\nFinal dummy test LOSS = {loss.item():.6f}")


Loss cross entropy: 0.009174452163279057
Loss BINARY cross entropy (object_mask): 0.04858735203742981
LOSS TOTAL: 0.1063491553068161

Final dummy test LOSS = 0.106349
